# 02_modelo_mlp.ipynb – Entrenamiento Red Neuronal

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
print("GPUs visibles para TensorFlow:", tf.config.list_physical_devices('GPU'))
print("CPUs visibles para TensorFlow:", tf.config.list_physical_devices('CPU'))



2025-12-28 01:41:20.616687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPUs visibles para TensorFlow: []
CPUs visibles para TensorFlow: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2025-12-28 01:41:24.720070: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-12-28 01:41:24.720098: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:160] env: CUDA_VISIBLE_DEVICES="-1"
2025-12-28 01:41:24.720103: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-12-28 01:41:24.720107: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-12-28 01:41:24.720111: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: gino-iglesias-Nitro-ANV15-41
2025-12-28 01:41:24.720114: I external/local_xla/xla/strea

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [3]:
df = pd.read_csv("BBDD_FINAL_sintetica.csv")
print("Tamaño de la data:", df.shape)
df.head()

Tamaño de la data: (5000, 31)


,station,year,season,DBO,DQO,OD,TDS,arsenico,bicarbonato,cadmio,...,nitratos,pH,perifiton_H,perifiton_J,plomo,potasio,sodio,sulfatos,temperatura,zinc
0,CHILI-1,2021,HÃºmeda,6.048422,7.606639,7.555917,154.821863,2.108431,47.052596,0.192954,...,1.015771,8.157850,0.175476,0.119987,1.048643,3.937205,17.027566,46.444176,17.878928,20.443940
1,CHILI-4,2019,Seca,3.619453,14.814122,7.612445,304.857953,1.222365,72.855623,0.070407,...,4.919001,8.038437,0.645816,0.934393,4.073503,6.556775,37.343220,53.714599,17.832196,22.539360
2,CHILI-3,2022,Seca,1.260012,9.784913,8.378081,321.006559,1.814172,72.531431,1.299086,...,4.675352,8.325971,1.462021,0.585827,11.647421,5.699583,37.947816,56.772489,15.103917,66.097356
3,CHILI-2,2023,Seca,2.001775,12.667794,7.713475,330.967015,1.117722,79.038083,0.095896,...,4.884165,7.606297,1.090298,0.639052,7.557435,6.100635,37.021685,59.394286,18.384227,22.807829
4,CHILI-2,2023,Seca,1.464026,12.876578,7.688734,316.876894,1.130597,76.532327,0.098073,...,4.430358,7.570012,1.085216,0.659245,7.097784,6.421333,41.695123,51.572254,17.976359,26.571200


In [4]:
cols_drop = ["station", "year", "season"]
df = df.drop(columns=[col for col in cols_drop if col in df.columns])
print("Shape luego de limpiar:", df.shape)
print("Columnas actuales:", df.columns.tolist())

Shape luego de limpiar: (5000, 28)
Columnas actuales: ['DBO', 'DQO', 'OD', 'TDS', 'arsenico', 'bicarbonato', 'cadmio', 'calcio', 'cloruros', 'cobre', 'coliformes termotolerantes', 'conductividad', 'cromo', 'escherichia coli', 'macrobentos_H', 'macrobentos_J', 'magnesio', 'mercurio', 'nitratos', 'pH', 'perifiton_H', 'perifiton_J', 'plomo', 'potasio', 'sodio', 'sulfatos', 'temperatura', 'zinc']


In [5]:
df.shape

(5000, 28)

In [19]:
target_col = "OD"

X = df.drop(columns=[target_col])
y = df[target_col]

feature_names = X.columns.tolist()

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", feature_names)

X shape: (5000, 27)
y shape: (5000,)
Feature names: ['DBO', 'DQO', 'TDS', 'arsenico', 'bicarbonato', 'cadmio', 'calcio', 'cloruros', 'cobre', 'coliformes termotolerantes', 'conductividad', 'cromo', 'escherichia coli', 'macrobentos_H', 'macrobentos_J', 'magnesio', 'mercurio', 'nitratos', 'pH', 'perifiton_H', 'perifiton_J', 'plomo', 'potasio', 'sodio', 'sulfatos', 'temperatura', 'zinc']


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train:", X_train.shape, " Test:", X_test.shape)

Train: (4000, 27)  Test: (1000, 27)


In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.summary()

/home/gino-iglesias/miniconda3/envs/unalm_ia/lib/python3.10/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,441 (5.63 KB)

 Trainable params: 1,441 (5.63 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,        # antes estaba en 10
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled, y_train,
    epochs=100,        # antes 200, con early_stopping sobra
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 35.5352 - mae: 5.6811 - val_loss: 13.4418 - val_mae: 3.2507
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.9604 - mae: 2.1803 - val_loss: 3.5624 - val_mae: 1.5173
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.2300 - mae: 1.1957 - val_loss: 1.4325 - val_mae: 0.9394
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9796 - mae: 0.7572 - val_loss: 0.7805 - val_mae: 0.6692
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5677 - mae: 0.5574 - val_loss: 0.4972 - val_mae: 0.5021
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3698 - mae: 0.4343 - val_loss: 0.3341 - val_mae: 0.4060
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2583 - mae: 0.3659 - val_loss: 0.2453 - val_mae: 0.3488
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1939 - mae: 0.3208 - val_loss: 0.1960 - val_mae: 0.3209
Epoch 9/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1m

In [13]:
mse, mae = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\n Evaluación en test -> MSE: {mse:.4f} | MAE: {mae:.4f}")


 Evaluación en test -> MSE: 0.0231 | MAE: 0.1214


In [17]:
import os
os.makedirs("../models", exist_ok=True)

model_path = "modelo_calidad_agua.h5"
model.save(model_path)
print(f"Modelo guardado")

Modelo guardado


In [20]:
import pickle
scaler_path = "scaler_features.pkl"
with open(scaler_path, "wb") as f:
    pickle.dump({"scaler": scaler, "features": feature_names}, f)

print(f" Scaler guardado")

 Scaler guardado


In [21]:
y_pred = model.predict(X_test_scaled)
print("Ejemplo de predicciones (primeros 5):")
for real, pred in list(zip(y_test.values[:5], y_pred[:5].flatten())):
    print(f"Real: {real:.3f}   |   Predicho: {pred:.3f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Ejemplo de predicciones (primeros 5):
Real: 7.556   |   Predicho: 7.491
Real: 7.513   |   Predicho: 7.609
Real: 7.765   |   Predicho: 7.499
Real: 7.048   |   Predicho: 6.911
Real: 7.747   |   Predicho: 7.714
